# Importing helper libraries

In [3]:
from fosforio import snowflake
from fosforml import *
from fosforml.constants import MLModelFlavours
from fosforio import get_dataframe
from matplotlib import pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
import warnings; warnings.simplefilter('ignore')
from joblib import dump, load
import requests
from tqdm import tqdm
import time
import calendar

from time import sleep
import configparser
from dateutil.relativedelta import relativedelta
import datetime
from dateutil.easter import easter
from scipy.optimize import minimize_scalar
from scipy.optimize import curve_fit
import holidays

%matplotlib inline

ModuleNotFoundError: No module named 'fosforml'

In [2]:
from scipy.optimize import curve_fit

# Define the demand curve function
def demand_curve(x, a, b, c, d, max_demand):
    demand = a * np.exp(-b * x) + c
    demand = np.where(x <= max_demand, np.minimum(demand, max_demand), demand)
    return demand + d

In [4]:
def revenue(price):
    return price * demand_curve(price, a_fit, b_fit,c_fit,d_fit,max_demand)

In [5]:
from scipy.optimize import brentq

def demand_to_price(num_rooms, a, b, c, d, max_demand):
    def root_func(x):
        return num_rooms - (a * np.exp(-b * x) + c)

    try:
        price = brentq(root_func, 0, 200)  # Adjust the interval bounds as needed
    except ValueError:
        # Fallback to default price if no root is found
        price_range=(0, 200)
        price = np.random.uniform(*price_range)

    return price

# Reading the data

In [2]:
snowflake.get_connection(connection_name="TTH_REV_OPT_CXN")
data = get_dataframe("BOOKINGS_TRANSFORMED")
data

Connection object created: <snowflake.connector.connection.SnowflakeConnection object at 0x7ffb5c53a040>
Please close the connection after use!
Reading dataframe from snowflake native connector


,ARRIVAL_DATE_TRANSFORMED,RESERVATION_STATUS_DATE_TRANSFORMED,HOTEL,IS_CANCELED,LEAD_TIME,ARRIVAL_DATE_YEAR,MONTH,ARRIVAL_DATE_WEEK_NUMBER,ARRIVAL_DATE_DAY_OF_MONTH,EXPECTED_ARRIVAL_DATE,RESERVATION_STATUS,RESERVATION_STATUS_DATE,TOTAL_STAY_NIGHTS,TALLY_DAYS,STAYS_IN_WEEKEND_NIGHTS,STAYS_IN_WEEK_NIGHTS,ADULTS,CHILDREN,BABIES,TOTAL_GUESTS,AVG_ROOMS_PER_NIGHT,TOTAL_ROOM_NIGHTS,MEAL,COUNTRY,MARKET_SEGMENT,DISTRIBUTION_CHANNEL,PREVIOUS_CANCELLATIONS,PREVIOUS_BOOKINGS_NOT_CANCELED,RESERVED_ROOM_TYPE,ASSIGNED_ROOM_TYPE,DEPOSIT_TYPE,DAYS_IN_WAITING_LIST,CUSTOMER_TYPE,ADR,TRANSCATION_IDS,CUST_ID,ROOM_LIMIT
0,2023-08-03,2023-08-06,City Hotel,0,86,2023,8,31,3,03-08-2023,Check.Out,06-08-2023,3,3.0,0,3,3,0,0,3,1,3,BB,DEU,Online TA,TA/TO,0,0,D,D,No Deposit,0,Transient,195.00,B20230803343215,C0106952,135
1,2023-08-02,2023-08-06,Resort Hotel,0,10,2023,8,31,2,02-08-2023,Check.Out,06-08-2023,4,4.0,0,4,2,0,0,2,1,4,BB,PRT,Online TA,TA/TO,0,0,A,A,No Deposit,0,Transient,228.50,B20230802911470,C0106953,360
2,2023-08-03,2023-08-06,City Hotel,0,180,2023,8,31,3,03-08-2023,Check.Out,06-08-2023,3,3.0,0,3,2,0,0,2,1,3,BB,GBR,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,80.10,B20230803481168,C0106954,460
3,2023-07-29,2023-08-06,City Hotel,0,148,2023,7,30,29,29-07-2023,Check.Out,06-08-2023,8,8.0,2,6,2,0,0,2,1,8,SC,FRA,Online TA,TA/TO,0,0,A,A,No Deposit,0,Transient,121.88,B20230729335744,C0106955,460
4,2023-08-05,2023-08-06,City Hotel,0,0,2023,8,31,5,05-08-2023,Check.Out,06-08-2023,1,1.0,0,1,1,0,0,1,1,1,SC,FRA,Online TA,TA/TO,0,0,A,A,No Deposit,0,Transient,179.00,B20230805458332,C0106956,460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109706,2023-08-04,2023-08-06,City Hotel,0,11,2023,8,31,4,04-08-2023,Check.Out,06-08-2023,2,2.0,0,2,2,0,0,2,1,2,SC,SLV,Online TA,TA/TO,0,0,A,A,No Deposit,0,Transient,160.00,B20230804347306,C0106947,460
109707,2023-07-30,2023-08-06,Resort Hotel,0,300,2023,7,31,30,30-07-2023,Check.Out,06-08-2023,7,7.0,2,5,2,0,0,2,1,7,BB,PRT,Online TA,TA/TO,0,0,A,A,No Deposit,0,Transient,149.81,B20230730168488,C0106948,360
109708,2023-07-29,2023-08-06,City Hotel,0,69,2023,7,30,29,29-07-2023,Check.Out,06-08-2023,8,8.0,2,6,2,0,0,2,1,8,SC,RUS,Online TA,TA/TO,0,0,A,A,No Deposit,0,Transient.Party,126.00,B20230729444159,C0106949,460
109709,2023-08-01,2023-08-06,City Hotel,0,147,2023,8,31,1,01-08-2023,Check.Out,06-08-2023,5,5.0,0,5,3,0,0,3,1,5,BB,SWE,Online TA,TA/TO,0,0,D,D,No Deposit,0,Transient,175.50,B20230801923755,C0106950,135


In [ ]:
def generate_holiday_dates(start_year, end_year):
    holidays = {}
    for year in range(start_year, end_year + 1):
        holidays[datetime.date(year, 1, 1)] = 'new_year'
        easter_date = easter(year)
        holidays[easter_date] = 'easter'
        holidays[datetime.date(year, 12, 25)] = 'christmas'
    return holidays

holidays = generate_holiday_dates(2020, 2023)

In [ ]:
def format_data(data):
    hotels = data['hotel'].unique()
    room_types = data['reserved_room_type'].unique()
    data['total_rns'] = data['stays_in_week_nights'] + data['stays_in_weekend_nights']
    data_to_transform = data[['hotel','reserved_room_type','arrival_date_transformed','total_rns','adr', 'room_limit']]
    expanded_df = pd.DataFrame()

    for _, row in data.iterrows():
        num_stay_dates = row['total_rns']
        try:
            # Create a row for each stay date
            expanded_booking = pd.DataFrame({
                'hotel': row['hotel'],
                'room_type': row['reserved_room_type'], 
                'arrival_date': pd.date_range(start=row['expected_arrival_date'], periods=num_stay_dates),
                'total_rns': 1,
                'adr': row['adr'],
                'room_limit': row['room_limit']
            })

            # Append the stay date information to the new dataframe
            expanded_df = pd.concat([expanded_df, expanded_booking], ignore_index=True)
        except ValueError as e:
            print(f"Error processing booking for {row['hotel']} on {row['expected_arrival_date']} : {num_stay_dates} {e}")
        
    expanded_df = expanded_df.sort_values('arrival_date')
    expanded_df = expanded_df.reset_index(drop=True)
    expanded_df['adr']= np.round(expanded_df['adr'], 2)
    holiday_dates = holidays.CountryHoliday('PT', years=[2020,2021,2022,2023])
    holidays = {
        expected_arrival_date: name
        for expected_arrival_date, name in holiday_dates.items()
        if name in ['Ano Novo', 'Páscoa', 'Dia de Natal']
    }
    expanded_df = expanded_df.rename({'Ano Novo':'new_year','Páscoa':'easter','Dia de Natal':'christmas'},axis=1)
    
    holidays = generate_holiday_dates(2020, 2023)
    
    # Define pre and post ranges for each holiday
    pre_range_offset = {'new_year': relativedelta(days=-1),
                        'easter': relativedelta(days=-2),
                        'christmas': relativedelta(days=-3)}

    post_range_offset = {'new_year': relativedelta(days=1),
                         'easter': relativedelta(days=2),
                         'christmas': relativedelta(days=3)}
    
    # Create new columns for each holiday
    for holiday in holidays.values():
        expanded_df[holiday] = 0

     # Set the holiday columns to 1 for matching dates
    for arrival_date, name in holidays.items():
        expanded_df.loc[expanded_df['arrival_date'] == arrival_date, name] = 1

        # Set the holiday columns to 1 for pre and post dates
        pre_offset = pre_range_offset.get(name)
        if pre_offset:
            pre_date = pd.to_datetime(arrival_date) + pre_offset
            expanded_df.loc[expanded_df['arrival_date'] == pre_date.strftime('%Y-%m-%d'), name] = 1

        post_offset = post_range_offset.get(name)
        if post_offset:
            post_date = pd.to_datetime(arrival_date) + post_offset
            expanded_df.loc[expanded_df['arrival_date'] == post_date.strftime('%Y-%m-%d'), name] = 1
            
    expanded_df['dow'] = expanded_df.arrival_date.dt.strftime('%A')
    expanded_df['month'] = expanded_df.arrival_date.dt.strftime('%B')
    
    non_holidays = expanded_df[expanded_df[['new_year', 'easter', 'christmas']].sum(axis=1) == 0]
    daily_rns= non_holidays.groupby(['arrival_date','dow','month', 'hotel', 'room_type']).agg({'room_limit': 'mean', 'total_rns':'sum'}).reset_index() # ge total stays per day

    daily_rns = daily_rns.groupby(['dow','month', 'hotel', 'room_type']).agg({'room_limit': 'mean','total_rns':['sum','mean','median']}).reset_index() # get Rns metrics by Dow & Month

    daily_rns.columns = ['_'.join(col) for col in daily_rns.columns] #remove multi level column
    adr_frequency = non_holidays.groupby(['dow','month','adr', 'hotel', 'room_type']).agg({'room_limit': 'mean','total_rns':'sum'})
    adr_frequency.reset_index(inplace=True)
    merged_df = pd.merge(adr_frequency, daily_rns,how='left',left_on=['dow','month', 'hotel', 'room_type'], right_on=['dow_','month_', 'hotel_', 'room_type_'],suffixes=('_act', '_tot'))

    merged_df = merged_df.drop(['dow_','month_'],axis=1)

    merged_df['probability'] = merged_df['total_rns']/merged_df['total_rns_sum']
    merged_df['expected_rns'] = merged_df['probability'] * merged_df['total_rns_median']
    merged_df = merged_df.sort_values(by=['dow', 'month', 'adr'], ascending=[True, True, False])
    merged_df['expected_demand']=merged_df.groupby(['dow', 'month'])['expected_rns'].cumsum()
    merged_df['expected_rev'] = merged_df['adr']* merged_df['expected_demand']
    results = pd.DataFrame(columns=['month', 'hotel','room_limit', 'room_type', 'dow', 'optimal_rate', 'expected_rn','expected_rev','optimal_rate_lim_inv'])
    months = merged_df.month.unique()
    dow = merged_df.dow.unique()
    for hotel in hotels:
        for room_type in room_types:
            for month in months:
                for day in dow:
                    # Get data for the specific combination
                    data_subset = merged_df[(merged_df['dow'] == day) & 
                                            (merged_df['hotel'] == hotel) & 
                                            (merged_df['room_type'] == room_type) & 
                                            (merged_df['month'] == month)].reset_index()

                    if data_subset.empty:
                        continue

                    # Remove outliers
                    mean = data_subset['adr'].mean()
                    std_dev = data_subset['adr'].std()
                    data_subset['z_scores'] = np.abs((data_subset['adr'] - mean) / std_dev)
                    data_subset = data_subset[data_subset['z_scores'] <= 2]

                    # Fit demand curve
                    x_data = data_subset['adr'].values
                    y_data = data_subset['expected_demand'].values

                    try:
                        initial_guess = [1, 0.01, 1, 1, data_subset['total_rns_median'].values[0]]
                        bounds = ([0, 0, 0, 0, 0], [np.inf, np.inf, np.inf, np.inf, np.inf])
                        maxfev = 10000  # Increase the number of maximum function evaluations
                        params, _ = curve_fit(demand_curve, x_data, y_data, bounds=bounds, p0=initial_guess, maxfev=maxfev)
                    except RuntimeError as e:
                        print(f"Error fitting demand curve for {hotel}, {room_type}, {month}, {day}: {e}")
                        continue

                    a_fit, b_fit, c_fit, d_fit, max_demand = params

                    # Optimize revenue
                    def revenue(price):
                        return price * demand_curve(price, a_fit, b_fit, c_fit, d_fit, max_demand)

                    objective = lambda price: -revenue(price)
                    optimize = minimize_scalar(objective, bounds=(45, 200), method='bounded')
                    optimal_price = optimize.x
                    max_revenue = -optimize.fun
                    expected_rns = demand_curve(optimal_price, a_fit, b_fit, c_fit, d_fit, max_demand)

                    optimal_rate_lim_inv = demand_to_price(data_subset['room_limit'].mean(), a_fit, b_fit, c_fit, d_fit, max_demand)

                    new_row = pd.DataFrame({'hotel': hotel,
                                            'room_type': room_type,
                                            'room_limit': data_subset['room_limit'].mean(),
                                            'month': month,
                                            'dow': day,
                                            'optimal_rate': optimal_price,
                                            'expected_rev': max_revenue,
                                            'expected_rn': expected_rns,
                                            'optimal_rate_lim_inv': optimal_rate_lim_inv}, index=[0])
                    results = pd.concat([results, new_row], ignore_index=True)
    results['optimal_rate'] = results['optimal_rate'].round()
    results['optimal_rate_lim_inv'] = results['optimal_rate_lim_inv'].round()

    results['expected_rn'] = results['expected_rn'].round().astype(int)
    results['expected_rev'] = results['expected_rev'].round()
    holidays =  expanded_df[expanded_df[['new_year', 'easter', 'christmas']].sum(axis=1) != 0]
    unpivoted = pd.melt(holidays, id_vars=['arrival_date', 'total_rns', 'adr', 'dow', 'month', "hotel", "room_type", 'room_limit'],
                        value_vars=['new_year', 'easter', 'christmas'],
                        var_name='holiday', value_name='holiday_indicator')
    unpivoted[unpivoted.holiday =='christmas']
    unpivoted = unpivoted[unpivoted['holiday_indicator'] == 1]
    holiday_adr = unpivoted.groupby(['holiday','adr','holiday_indicator', "hotel", "room_type"]).agg({'room_limit': 'mean', 'total_rns':'sum'}).reset_index()
    holiday_rns = unpivoted.groupby(['arrival_date','holiday', "hotel", "room_type"]).agg({'room_limit': 'mean','total_rns':'sum'}).reset_index()
    holiday_rns = holiday_rns.groupby(['holiday', "hotel", "room_type"]).agg({'room_limit': 'mean', 'total_rns':['sum','mean','median']}).reset_index()
    holiday_rns.columns = ['_'.join(col) for col in holiday_rns.columns]
    merged_holidays = pd.merge(holiday_adr, holiday_rns,how='left',left_on=['holiday' ,'hotel', 'room_type'], right_on=['holiday_' ,'hotel_', 'room_type_'],suffixes=('_act', '_tot'))

    merged_holidays.drop('holiday_',axis=1,inplace=True)
    merged_holidays['probability'] = merged_holidays['total_rns']/merged_holidays['total_rns_sum']
    merged_holidays['expected_rns'] = merged_holidays['probability'] * merged_holidays['total_rns_median']
    merged_holidays = merged_holidays.sort_values(by=['holiday', 'adr'], ascending=[True, False])
    merged_holidays['expected_demand']=merged_holidays.groupby(['holiday'])['expected_rns'].cumsum()
    unique_holidays = merged_holidays.holiday.unique()

    
    holiday_results = pd.DataFrame(columns=['holiday', 'optimal_rate', 'expected_rn','expected_rev','optimal_rate_lim_inv'])

    for hotel in hotels:
        for room_type in room_types:

            for day in unique_holidays:
                    data = merged_holidays[(merged_holidays.holiday == day) & (merged_holidays.hotel == hotel) & (merged_holidays.room_type == room_type) ].reset_index()

                    #remove outlier
                    mean = data.adr.mean()
                    std_dev = data.adr.std()


                    # calculate z-scores
                    data['z_scores'] = np.abs((data.adr - mean) / std_dev)

                    #filter out outliers
                    data = data[data.z_scores <=2]

                    ## Fit Demand curve
                    x_data = data['adr'].values
                    y_data = data['expected_demand'].values

                    initial_guess = [1, 0.01, 1, 1,data['total_rns_median'].values[0] ]
                    bounds = ([0, 0, 0, 0, 0], [np.inf, np.inf, np.inf, np.inf, np.inf])

                    try:
                        params, _ = curve_fit(demand_curve, x_data, y_data, bounds=bounds, p0=initial_guess)
                    except Exception as e:
                        print(f"Error fitting demand curve for {hotel}, {room_type}: {e}")
                        continue

                    # Extract the fitted parameters
                    a_fit, b_fit, c_fit ,d_fit,max_demand= params

                    a_fit, b_fit, c_fit ,d_fit,max_demand = np.round(a_fit, 3), np.round(b_fit, 3), np.round(c_fit, 3) ,np.round(d_fit, 3),np.round(max_demand, 3)
                    #optimze revenue
                    objective = lambda price: -revenue(price)



                    optimize = minimize_scalar(objective, bounds=(45, 400), method='bounded')
                    optimal_price = optimize.x
                    max_revenue = -optimize.fun

                    expected_rns = demand_curve(optimal_price,a_fit, b_fit, c_fit ,d_fit,max_demand)
                    print(50,a_fit,b_fit,c_fit,d_fit,max_demand)
                    optimal_rate_lim_inv = demand_to_price(data.room_limit.mean() ,a_fit,b_fit,c_fit,d_fit,max_demand)

                    new_row = pd.DataFrame({'holiday':day,
                                            'hotel': hotel,
                                            'room_limit': data.room_limit.mean(),
                                   'room_type': room_type,
                                   'optimal_rate': optimal_price,
                                   'expected_rev':max_revenue,
                                   'expected_rn':expected_rns,
                                   'optimal_rate_lim_inv':optimal_rate_lim_inv},index=[0])
                    holiday_results = pd.concat([holiday_results, new_row], ignore_index=True)
    years = [2020, 2021, 2022, 2023]

    holiday_dates = []
    for year in years:
        for index, row in holiday_results.iterrows():
            if row['holiday'] == 'christmas':
                date = datetime.date(year, 12, 25)
            elif row['holiday'] == 'easter':
                date = easter(year)
            elif row['holiday'] == 'new_year':
                date = datetime.date(year, 1, 1)

            holiday_dates.append({
                'hotel': row['hotel'],
                'room_type': row['room_type'],
                'month': date.strftime('%B'),
                'dow': date.strftime("%A"),
                'holiday': row['holiday'],
                'optimal_rate': row['optimal_rate'],
                'expected_rn': row['expected_rn'],
                'expected_rev': row['expected_rev'],
                'optimal_rate_lim_inv': row['optimal_rate_lim_inv'],
                'arrival_date': pd.to_datetime(date)
            })

    holiday_results_yearly = pd.DataFrame(holiday_dates)

    
hotel_types = ['Resort Hotel', 'City Hotel']
room_types = ['A', 'D', 'E']

combinations = list(itertools.product(hotel_types, room_types))

combinations

combinations_df = pd.DataFrame(combinations, columns=['hotel', 'room_type'])
combinations_df